In [17]:
import psycopg2
import pandas as pd
# from sqlalchemy import create_engine
# from sqlalchemy.orm import Session

''' Для подавления сообщений об ошибках при обработке pandas ответов psycopg2'''
import warnings
warnings.filterwarnings('ignore')

In [18]:
''' Аттрибуты подключения содержатся в отдельном файле для безопасности.
    Импортируются в словарь, который далее используется при подключении
    и запросах к базе.
'''
import creds as c_

CONN_ATTR = {
    'user': c_.USER,
    'password': c_.PASSWORD,
    'host': c_.HOST,
    'port': c_.PORT,
    'dbname': c_.DBNAME
    }

def make_request(query_: str) -> object:
    ''' Делает запросы к БД с определёнными аттрибутами
    Arguments:
        query_ [str] -- Строка SQL-запроса (можно многострочник)
    Returns:
        [object] -- Ответ на запрос
    '''
    with psycopg2.connect(**CONN_ATTR) as conn:
        return pd.read_sql_query(query_, conn)

In [19]:
CONN_STR = f'postgresql+psycopg2:///{c_.USER}:{c_.PASSWORD}@{c_.HOST}:{c_.PORT}/{c_.DBNAME}'

def make_request_sa1(query_):
    with create_engine(CONN_STR) as conn:
        return pd.read_sql_query(query_, conn)

def make_request_sa2(query_):
    ENGINE = create_engine(CONN_STR)
    with ENGINE.connect() as conn:
        output_ = pd.read_sql_query(query_, conn.execute(text(query_)))
        # conn.close()
        return output_
def make_request_sa3(query_):
    conn = create_engine(CONN_STR)
    output_ = pd.read_sql_query(query_, conn)
    # conn.close_all()
    return output_


### Задание 3.1 ###

Сколько вакансий есть в базе?

***Ответ - 49197***

In [20]:
query_3_1 = f'''
SELECT COUNT(*)
FROM public.vacancies
'''
display(make_request(query_3_1))

,count
0,49197


### Задание 3.2 ###

Сколько работодателей?

***Ответ - 23501***

In [21]:
query_3_2 = f'''
SELECT COUNT(*)
FROM public.employers
'''
display(make_request(query_3_2))

,count
0,23501


### Задание 3.3 ###

Сколько регионов?

***Ответ - 1362***

In [22]:
query_3_2 = f'''
SELECT COUNT(*)
FROM public.areas
'''
display(make_request(query_3_2))

,count
0,1362


### Задание 3.3 ###

Сколько сфер деятельности?

***Ответ - 294***

In [23]:
query_3_4 = f'''
SELECT COUNT(*)
FROM public.industries
'''
display(make_request(query_3_4))

,count
0,294


### Задание 4.1 ###

Написать запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Выбрать пятёрку лидеров по количеству вакансий:

***Ответ - Москва, Санкт-Петербург, Минск, Новосибирск, Алматы***

In [24]:
query_4_1 = f'''
SELECT
    a.name AS area,
    COUNT(*) AS cnt
FROM public.vacancies AS v
JOIN public.areas AS a ON
    v.area_id = a.id
GROUP BY a.id
ORDER BY cnt DESC
LIMIT 10
'''
display(make_request(query_4_1))

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


### Задание 4.2 ###

У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

***Ответ - 24073***

In [25]:
query_4_2 = f'''
SELECT
    COUNT(*)
FROM public.vacancies
WHERE
    salary_from IS NOT NULL
    OR
    salary_to IS NOT NULL
'''
display(make_request(query_4_2))

,count
0,24073


### Задание 4.3 ###

Найти средние значения для нижней и верхней границы зарплатной вилки. Округлить значения до целого числа.

***Ответ - 71065, 110537***

In [26]:
query_4_3 = f'''
SELECT
    AVG(salary_from)::integer AS avg_min,
    AVG(salary_to)::integer AS avg_max
FROM public.vacancies
'''
display(make_request(query_4_3))

,avg_min,avg_max
0,71065,110537


### Задание 4.4 ###

Написать запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Какая пара находится на втором месте по популярности?

***Ответ - Удалённая работа + Полная занятость***

In [27]:
query_4_4 = f'''
SELECT
    schedule,
    employment,
    COUNT(*) AS cnt
FROM public.vacancies
GROUP BY schedule, employment
ORDER BY cnt DESC
OFFSET 1
LIMIT 1
'''
display(make_request(query_4_4))

,schedule,employment,cnt
0,Удаленная работа,Полная занятость,7802


### Задание 4.5 ###

Написать запрос, выводящий значения поля **Требуемый опыт работы** (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

***Ответ - Более 6 лет / Нет опыта / От 3 до 6 лет / От 1 года до 3 лет***

In [28]:
query_4_5 = f'''
SELECT
    experience AS "Требуемый опыт работы"
FROM public.vacancies
GROUP BY experience
ORDER BY COUNT(*)
'''
display(make_request(query_4_5))

,Требуемый опыт работы
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


### Задание 5.1 ###

Написать запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

***Ответ - Яндекс / Газпром нефть***

In [29]:
query_5_1 = f'''
SELECT
    e.name
FROM public.vacancies AS v
JOIN public.employers AS e ON
    v.employer_id = e.id
GROUP BY e.name
ORDER BY COUNT(*) DESC
LIMIT 5
'''
display(make_request(query_5_1))

,name
0,Яндекс
1,Ростелеком
2,Тинькофф
3,СБЕР
4,Газпром нефть


### Задание 5.2 ###

Написать запрос, который для каждого региона выведет количество работодателей и вакансий в нём.

Среди регионов, в которых нет вакансий, найти тот, в котором наибольшее количество работодателей. Вписать его название в поле ниже в том виде, который вернул запрос.

***Ответ - Россия***

In [50]:
query_5_2 = f'''
SELECT
    a.name,
    COUNT(DISTINCT e.id) AS employers_cnt,
    COUNT(DISTINCT v.id) AS vacancies_cnt
FROM public.areas AS a
LEFT JOIN public.employers AS e ON
    a.id = e.area
LEFT JOIN public.vacancies AS v ON
    a.id = v.area_id
GROUP BY a.id
ORDER BY vacancies_cnt, employers_cnt DESC
LIMIT 1
'''
display(make_request(query_5_2))

,name,employers_cnt,vacancies_cnt
0,Россия,410,0


In [55]:
query_5_2_1 = f'''
SELECT
    a.name,
    COUNT(DISTINCT e.id) AS employers_cnt,
    COUNT(DISTINCT v.id) AS vacancies_cnt
FROM public.areas AS a
LEFT JOIN public.employers AS e ON
    a.id = e.area
LEFT JOIN public.vacancies AS v ON
    a.id = v.area_id
GROUP BY a.id
ORDER BY vacancies_cnt, employers_cnt DESC
OFFSET 19
LIMIT 20
'''
display(make_request(query_5_2_1))

,name,employers_cnt,vacancies_cnt
0,Австралия,6,0
1,Кировская область,6,0
2,Швейцария,5,0
3,Ивановская область,5,0
4,Канада,5,0
5,Тульская область,5,0
6,Новосибирская область,5,0
7,Липецкая область,5,0
8,Иркутская область,4,0
9,Пензенская область,4,0


### Задание 5.3 ###

Для каждого работодателя посчитать количество регионов, в которых он публикует свои вакансии.

Выбрать максимальное значение из получившегося списка.

***Ответ - 181***

In [31]:
query_5_3 = f'''
SELECT
    e.name,
    COUNT(DISTINCT v.area_id) AS a_cnt
FROM public.employers AS e
LEFT JOIN public.vacancies AS v ON
    e.id = v.employer_id
GROUP BY e.id
ORDER BY a_cnt DESC
LIMIT 1
'''
display(make_request(query_5_3))

,name,a_cnt
0,Яндекс,181


### Задание 5.4 ###

Написать запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

***Ответ - 8419***

In [32]:
query_5_4 = f'''
SELECT
    COUNT(DISTINCT e.id)
FROM public.employers AS e
LEFT JOIN public.employers_industries AS ei ON
    e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''
display(make_request(query_5_4))

,count
0,8419


###  Задание 5.5 ###

Написать запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

***Ответ - 2ГИС***

In [33]:
query_5_5 = f'''
SELECT
    e.name,
    COUNT(*)
FROM public.employers_industries AS ei
JOIN public.employers AS e ON
    e.id = ei.employer_id
GROUP BY e.id
HAVING COUNT(*) = 4
ORDER BY e.name
OFFSET 2
LIMIT 1
'''
display(make_request(query_5_5))

,name,count
0,2ГИС,4


###  Задание 5.6 ###

С помощью запроса выяснить, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

***Ответ - 3553***

In [34]:
query_5_6 = f'''
SELECT
    COUNT(e.id)
FROM public.employers AS e
JOIN public.employers_industries AS ei ON
    e.id = ei.employer_id
JOIN public.industries AS i ON
    i.id = ei.industry_id
    AND
    i.name = 'Разработка программного обеспечения'
'''
display(make_request(query_5_6))

,count
0,3553


###  Задание 5.7 ###

Для компании «Яндекс» вывести список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавить строку **Total** с общим количеством вакансий компании. Должна получиться выборка такого вида (приведён пример результата для компании **SberTech**):

***Ответ - 17 строк, Total = 485***

In [35]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
wiki_url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

response = requests.get(wiki_url)
page = bs(response.text, 'html.parser')
megas_list = []
tag_list = page.find('table', class_='standard sortable').find_all('a')
for tag_ in tag_list:
    str_ = tag_.text
    if str_[0].isupper():
        megas_list.append(str_)
megas_str = "('" + "', '".join(megas_list) + "')"
print(megas_str)


('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [36]:
query_5_7 = f'''
(
    SELECT
        a.name,
        COUNT(v.id) AS cnt
    FROM public.vacancies AS v
    JOIN public.employers AS e ON
        e.id = v.employer_id
        AND
        e.name = 'Яндекс'
    JOIN public.areas AS a ON
        a.id = v.area_id
        AND
        a.name IN {megas_str}
    GROUP BY a.name
)

UNION ALL

(
    SELECT
        'Total',
        COUNT(*)
    FROM public.vacancies AS vv
    JOIN public.employers AS ee ON
        ee.id = vv.employer_id
        AND
        ee.name = 'Яндекс'
    JOIN public.areas AS aa ON
        aa.id = vv.area_id
        AND
        aa.name IN {megas_str}
)
ORDER BY cnt
'''
display(make_request(query_5_7))

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


###  Задание 6.1 ###

Сколько вакансий имеет отношение к данным?

***Ответ - 1771***

In [37]:
query_6_1 = f'''
SELECT
    COUNT(name)
FROM public.vacancies
WHERE
    LOWER(name) LIKE '%data%'
    OR
    LOWER(name) LIKE '%данн%'
'''
display(make_request(query_6_1))

,count
0,1771


### Задание 6.2 ###

Сколько есть подходящих вакансий для начинающего дата-сайентиста?

Считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:

    'data scientist';
    'data science';
    'исследователь данных';
    'ML' (здесь не нужно брать вакансии по HTML);
    'machine learning';
    'машинн%обучен%'.

В следующих заданиях продолжитсяработа с вакансиями по указанному выше условию (уже не учитывая вакансии уровня Junior).

Считать вакансиями для специалистов уровня Junior следующие:

    в названии есть слово 'junior'
    или требуемый опыт — "Нет опыта"
    или тип трудоустройства — "Стажировка".

В качестве ответа записать количество вакансий, которое вернул запрос.

***Ответ - 51***


In [42]:
query_6_2 = f'''
SELECT
    COUNT(*)
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            LOWER(name) NOT LIKE '%html%'
            AND
            LOWER(name) LIKE '%ml%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND
    (
        LOWER(name) LIKE '%junior%'
        OR
        experience = 'Нет опыта'
        OR
        employment = 'Стажировка'
    )
'''
display(make_request(query_6_2))

,count
0,51


###  Задание 6.3 ###

Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

В качестве ответа записать количество, которое вернул запрос.

***Ответ - 201***

In [43]:
query_6_3 = f'''
SELECT
    COUNT(*)
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND
    (
        LOWER(key_skills) LIKE '%sql%'
        OR
        LOWER(key_skills) LIKE '%postgres%'
    )
'''
display(make_request(query_6_3))

,count
0,201


###  Задание 6.4 ###

С помощью запроса, аналогичного предыдущему, проверить, насколько популярен Python в требованиях работодателей к DS. Вычислить количество вакансий, в которых в качестве ключевого навыка указан Python.

В качестве ответа записать количество, которое вернул запрос.

***Ответ - 351***

In [46]:
query_6_4 = f'''
SELECT
    COUNT(*)
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND
    LOWER(key_skills) LIKE '%python%'
'''
display(make_request(query_6_4))

,count
0,351


###  Задание 6.5 ###

Сколько ключевых навыков в среднем указывают в вакансиях для DS?

Ответ округлить до двух знаков после точки-разделителя.

***Ответ - 6.41***

In [47]:
query_6_5 = f'''
SELECT
    -- Для вычисления в строке из "key_skills" ключевых навыков, разделённых
    -- табуляторами, надо из длины всей строки вычесть её же длину без табов
    -- плюс один
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1), 2) AS round_avg
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND key_skills IS NOT NULL
'''
display(make_request(query_6_5))

,round_avg
0,6.41


### Задание 6.6 ###

Написать запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).

При решении задачи принять во внимание следующее:

1. Рассматриваются только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, считать зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, его и считать зарплатой по вакансии.
3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотреть, что возвращает запрос **`select 1 + null`**).
4. Чтобы избежать этой ситуации, воспользоваться функцией `coalesce` , которая заменит `null` на значение, которое передадим. Например, посмотреть, что возвращает запрос **`select 1 + coalesce(null, 0)`**.

Выяснить на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлить до целого числа.

***Ответ - 243115***

In [48]:
query_6_6 = f'''
SELECT
    experience AS "Опыт",
    ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2)) AS "Средняя зарплата в DS"
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND
    (
        salary_from IS NOT NULL
        OR
        salary_to IS NOT NULL
    )
GROUP BY experience
'''
display(make_request(query_6_6))

,Опыт,Средняя зарплата в DS
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


----

# Дополнительные исследования #